# 03 · Modelado de clustering (activos)


In [ ]:
# =========================
# PASO 0 — CHECKLIST COHERENCIA (NB-03)
# =========================
# Objetivo: antes de auditar a fondo (Paso 1), verificar que los datasets de entrada
# cumplen lo acordado en NB-01 y NB-02 y están alineados para el clustering.

# ────────────────────────
# 1) Imports y configuración base de proyecto
# ────────────────────────
import sys
from pathlib import Path
from dotenv import load_dotenv
import os
import pandas as pd
import numpy as np

# Asegura que podamos importar módulos del repo si hiciera falta (src/, etc.)
ROOT = Path.cwd().parent  # carpeta raíz del proyecto (sube un nivel desde /notebooks)
if str(ROOT) not in sys.path:
    sys.path.append(str(ROOT))

# Carga variables de entorno (permite override de rutas sin tocar el código)
load_dotenv()

# Opcional: que no se corten columnas en displays
pd.set_option("display.max_columns", None)
pd.set_option("display.width", None)

# ────────────────────────
# 2) Resolución de rutas (con .env y defaults)
# ────────────────────────
# Patrón de nombres de variables de entorno (puedes cambiarlos si ya definiste otros):
# - X_BASE_PATH:              ruta a activos_for_model_v2.csv (baseline, sin Empresa)
# - X_EMPRESA_PATH:           ruta a activos_for_model_v2_empresa.csv (con Empresa)
# - X_IDS_PATH:               ruta a activos_ids_v2_plus.csv (IDs de reenganche)
#
# Defaults (si .env no define nada): en data/processed/ bajo ROOT.

DEFAULT_BASE = ROOT / "data" / "processed" / "activos_for_model_v2.csv"
DEFAULT_EMP = ROOT / "data" / "processed" / "activos_for_model_v2_empresa.csv"
DEFAULT_IDS = ROOT / "data" / "processed" / "activos_ids_v2_plus.csv"

X_BASE_PATH = Path(os.getenv("X_BASE_PATH", str(DEFAULT_BASE)))
X_EMP_PATH  = Path(os.getenv("X_EMPRESA_PATH", str(DEFAULT_EMP)))
X_IDS_PATH  = Path(os.getenv("X_IDS_PATH", str(DEFAULT_IDS)))

# ────────────────────────
# 3) Carga de datos con mensajes claros de error
# ────────────────────────
def _leer_csv_seguro(path: Path, nombre_log: str) -> pd.DataFrame:
    """Lee un CSV y lanza un error amigable si no existe o falla la lectura."""
    if not path.exists():
        raise FileNotFoundError(
            f"[{nombre_log}] No se encontró el archivo en: {path}\n"
            f"→ Revisa tu .env (X_*_PATH) o coloca el archivo en la ruta por defecto."
        )
    try:
        return pd.read_csv(path, low_memory=False)
    except Exception as e:
        raise RuntimeError(
            f"[{nombre_log}] Error al leer el CSV en {path}:\n{e}"
        )

df_base     = _leer_csv_seguro(X_BASE_PATH, "BASE (sin Empresa)")
df_empresa  = _leer_csv_seguro(X_EMP_PATH,  "CON EMPRESA")
df_ids      = _leer_csv_seguro(X_IDS_PATH,  "IDS")

print("RUTAS USADAS")
print("  Base (sin Empresa):", X_BASE_PATH)
print("  Con Empresa       :", X_EMP_PATH)
print("  IDs               :", X_IDS_PATH)
print()

# ────────────────────────
# 4) Checks principales (shapes y consistencia de filas)
# ────────────────────────
print("SHAPES")
print("  Base (sin Empresa):", df_base.shape)
print("  Con Empresa       :", df_empresa.shape)
print("  IDs               :", df_ids.shape)
print()

same_rows_be = (df_base.shape[0] == df_empresa.shape[0])
same_rows_bi = (df_base.shape[0] == df_ids.shape[0])
print("¿Mismo nº de filas Base vs Empresa?  ", same_rows_be)
print("¿Mismo nº de filas Base vs IDs?      ", same_rows_bi)
if not (same_rows_be and same_rows_bi):
    print("⚠️ Advertencia: los datasets no tienen el mismo nº de filas. Revisa el pipeline de NB-02.")
print()

# ────────────────────────
# 5) Columnas de presencias (deben ser EXACTAMENTE 4: 15d, 1m, 3m, 6m)
# ────────────────────────
pres_cols = [c for c in df_base.columns if c.endswith("_pres")]
print("Columnas de presencias detectadas en Base:", pres_cols)

esperadas_pres = {"Atencion15d_pres", "Atencion1m_pres", "Atencion3m_pres", "Atencion6m_pres"}
faltan_pres = esperadas_pres - set(pres_cols)
sobran_pres = set(pres_cols) - esperadas_pres

print("¿FALTAN presencias esperadas? ", faltan_pres if faltan_pres else "No")
print("¿SOBRAN presencias no esperadas?", sobran_pres if sobran_pres else "No")
print()

# ────────────────────────
# 6) Numéricas clave (KPIs y cantidades + edad)
# ────────────────────────
numericas_clave = [
    "Edad", "CantPptos", "CantPptosAbo", "CantPptosAvan",
    "TotPptos_l1p", "TotPptosAbo_l1p", "TotPptosAvan_l1p",
    "PctCumplimiento", "TicketPromPpto_l1p"
]
faltantes_num = [c for c in numericas_clave if c not in df_base.columns]
print("Numéricas clave faltantes en Base:", faltantes_num if faltantes_num else "Ninguna")
print()

# ────────────────────────
# 7) Columnas adicionales en el set CON EMPRESA (deben incluir Empresa/Convenio)
# ────────────────────────
cols_extra = list(set(df_empresa.columns) - set(df_base.columns))
# Orden alfabético solo para leer más fácil:
cols_extra = sorted(cols_extra)
ejemplos_esperados = ("TieneEmpresa", "EsConvenio", "Empresa_grp_")
print("Columnas adicionales (Con Empresa vs Base):", cols_extra[:15], ("… (+)" if len(cols_extra) > 15 else ""))
print("¿Incluye señales esperadas?",
      any(c == "TieneEmpresa" for c in cols_extra) and
      any(c == "EsConvenio" for c in cols_extra) and
      any(c.startswith("Empresa_grp_") for c in cols_extra))
print()

# ────────────────────────
# 8) NaNs totales (ideal: 0, según NB-02 imputación ya aplicada)
# ────────────────────────
nans_base = int(df_base.isna().sum().sum())
nans_emp  = int(df_empresa.isna().sum().sum())
print("NaNs totales en Base:   ", nans_base)
print("NaNs totales en Empresa:", nans_emp)
if nans_base > 0 or nans_emp > 0:
    print("⚠️ Advertencia: hay NaNs. Revisa imputación de NB-02 antes de escalar y clusterizar.")
print()

# ────────────────────────
# 9) (Opcional) Vista rápida de 5 filas para olfateo
# ────────────────────────
print("Muestra Base (5 filas):")
display(df_base.head())

print("Muestra Con Empresa (5 filas):")
display(df_empresa.head())

print("Muestra IDs (5 filas):")
display(df_ids.head())

# ────────────────────────
# 10) Mini-tests (asserts suaves con mensajes claros)
# ────────────────────────
def _ok(flag: bool) -> str:
    return "OK ✅" if flag else "FALLO ❌"

tests = {
    "FILAS_base_vs_empresa": same_rows_be,
    "FILAS_base_vs_ids"    : same_rows_bi,
    "PRES_existen_4"       : (set(pres_cols) == esperadas_pres),
    "NUM_no_faltantes"     : (len(faltantes_num) == 0),
    "EMP_columnas_extra"   : (
        any(c == "TieneEmpresa" for c in cols_extra) and
        any(c == "EsConvenio" for c in cols_extra) and
        any(c.startswith("Empresa_grp_") for c in cols_extra)
    ),
    "NANS_cero_base"       : (nans_base == 0),
    "NANS_cero_empresa"    : (nans_emp == 0),
}

print("\nRESULTADO MINI-TESTS")
for k, v in tests.items():
    print(f"  {k:22s}: {_ok(v)}")



In [ ]:
# ============================================================
# PASO 1 — AUDITORÍA DE DATOS (NB-03) — SOLO INSPECCIÓN
# ============================================================
# Objetivo: ver el "esquema real" de los datasets que usaremos en el clustering,
# SIN transformar nada. De este paso vamos a salir con listas de columnas
# (numéricas / presencias / dummies) listas para el pipeline del Paso 2.

# ────────────────────────────────────────────────────────────
# 1) Vista general (shape, columnas, tipos, nulos) — ambos sets
# ────────────────────────────────────────────────────────────
print("=== RUTAS USADAS ===")
print("Base (sin Empresa):", X_BASE_PATH)                                             # confirma ruta efectiva
print("Con Empresa       :", X_EMP_PATH)
print("IDs               :", X_IDS_PATH)
print()

print("=== SHAPES ===")
print("Base    :", df_base.shape)                                                     # filas x columnas baseline
print("Empresa :", df_empresa.shape)                                                  # filas x columnas con Empresa
print("IDs     :", df_ids.shape)                                                      # filas x columnas IDs
print()

print("=== TIPOS (dtypes) — Base (TOP 10) ===")
print(df_base.dtypes.sort_index().head(10))                                           # muestra 10 tipos para ver variedad
print("… (total columnas Base:", df_base.shape[1], ")")
print()

print("=== TIPOS (dtypes) — Con Empresa (TOP 10) ===")
print(df_empresa.dtypes.sort_index().head(10))                                        # idem para variante con Empresa
print("… (total columnas Empresa:", df_empresa.shape[1], ")")
print()

print("=== NULOS POR COLUMNA — Base (TOP 10 con más nulos) ===")
nul_base = df_base.isna().sum().sort_values(ascending=False)                          # cuenta nulos por columna
print(nul_base.head(10))                                                              # top 10 columnas más afectadas
print("Nulos totales Base:", int(nul_base.sum()))                                     # nulos totales
print()

print("=== NULOS POR COLUMNA — Con Empresa (TOP 10 con más nulos) ===")
nul_emp = df_empresa.isna().sum().sort_values(ascending=False)                        # idem para variante con Empresa
print(nul_emp.head(10))
print("Nulos totales Empresa:", int(nul_emp.sum()))
print()

print("=== MUESTRAS (5 filas) — Base / Empresa / IDs ===")
display(df_base.head(5))                                                               # vistazo rápido a filas
display(df_empresa.head(5))
display(df_ids.head(5))

# ────────────────────────────────────────────────────────────
# 2) Identificación de grupos de columnas para el pipeline
#    * No transformamos, solo mapeamos las listas que usaremos en Paso 2 *
# ────────────────────────────────────────────────────────────

# a) Presencias de atención (esperamos exactamente 4)
PRES_COLS = [c for c in df_base.columns if c.endswith("_pres")]                       # detecta columnas *_pres
PRES_ESPERADAS = ["Atencion15d_pres","Atencion1m_pres","Atencion3m_pres","Atencion6m_pres"]  # set acordado NB-02
print("\n=== PRESENCIAS DETECTADAS (Base) ===")
print(PRES_COLS)
if set(PRES_COLS) != set(PRES_ESPERADAS):                                             # avisa si hay diferencias
    print("⚠️ Ojo: diferencia con las esperadas:", set(PRES_ESPERADAS) ^ set(PRES_COLS))

# b) Numéricas clave (cantidades + montos log1p + KPIs + edad)
NUMERIC_CORE = [
    "Edad","CantPptos","CantPptosAbo","CantPptosAvan",                                # cantidades y edad
    "TotPptos_l1p","TotPptosAbo_l1p","TotPptosAvan_l1p",                              # montos log1p
    "PctCumplimiento","TicketPromPpto_l1p"                                            # KPIs
]
FALTAN_NUM = [c for c in NUMERIC_CORE if c not in df_base.columns]                    # verifica si falta alguna
print("\n=== NUMÉRICAS CLAVE (Base) ===")
print(NUMERIC_CORE)
print("Faltantes:", FALTAN_NUM if FALTAN_NUM else "Ninguna")

# c) Dummies de geografía (Comuna y Región)
DUM_COMUNA = [c for c in df_base.columns if c.startswith("Comuna_grp_")]              # todas las dummies de comuna
DUM_REGION = [c for c in df_base.columns if c.startswith("Region_")]                  # todas las dummies de región
print("\n=== DUMMIES GEO — Comuna (n=", len(DUM_COMUNA), ") ===", sep="")
print(DUM_COMUNA[:15], ("… (+)" if len(DUM_COMUNA) > 15 else ""))                     # muestra primeras 15
print("=== DUMMIES GEO — Región (n=", len(DUM_REGION), ") ===", sep="")
print(DUM_REGION)

# d) Señal Empresa/Convenio (solo existen en df_empresa)
EMP_BIN = [c for c in df_empresa.columns if c in ("TieneEmpresa","EsConvenio")]       # flags binarios
EMP_DUM = [c for c in df_empresa.columns if c.startswith("Empresa_grp_")]             # dummies top-N por empresa
print("\n=== EMPRESA/CONVENIO — Binarios ===")
print(EMP_BIN if EMP_BIN else "No detectados (esperado: en set con Empresa)")
print("=== EMPRESA/CONVENIO — Dummies (n=", len(EMP_DUM), ") ===", sep="")
print(EMP_DUM[:15], ("… (+)" if len(EMP_DUM) > 15 else ""))

# e) Otras columnas potencialmente binarias (0/1) útiles
#    *No son presencias ni geo/empresa, pero pueden ser señales tipo flag*
OTRAS_BIN = []
for c in df_base.columns:                                                             # iteramos todas las columnas
    if c in NUMERIC_CORE or c in PRES_COLS:                                           # excluimos numéricas y presencias
        continue
    if c.startswith("Comuna_grp_") or c.startswith("Region_"):                        # excluimos geo
        continue
    if c.endswith("_grp") or c.endswith("_cat"):                                      # excluimos posibles labels no dummies
        continue
    serie = df_base[c]
    # Detecta columnas con valores solo {0,1} (ignorando NaNs). Ojo con tipos object "0"/"1".
    vals = pd.Series(serie.dropna().unique()).astype(str)
    if set(vals).issubset({"0","1"}):                                                 # si solo hay 0/1 como strings
        OTRAS_BIN.append(c)

print("\n=== OTRAS BINARIAS 0/1 (Base) ===")
print(OTRAS_BIN if OTRAS_BIN else "Ninguna detectada adicional")

# f) Columnas de identificación / metadata (para NO usarlas como features)
ID_CANDIDATAS = []
for c in df_ids.columns:                                                               # cualquier ID del archivo IDs
    if c in df_base.columns:
        ID_CANDIDATAS.append(c)
print("\n=== POSIBLES IDs PRESENTES EN Base (para excluir de features) ===")
print(ID_CANDIDATAS if ID_CANDIDATAS else "No hay IDs del archivo IDs dentro de Base (ideal)")

# ────────────────────────────────────────────────────────────
# 3) Construcción de LISTAS FINALES de columnas por variante
#    * Estas listas se usarán en el Paso 2 para armar el ColumnTransformer *
# ────────────────────────────────────────────────────────────

# LISTAS PARA VARIANTE BASELINE (SIN EMPRESA)
NUM_COLS_BASELINE   = [c for c in NUMERIC_CORE if c in df_base.columns]               # numéricas presentes
PRES_COLS_BASELINE  = PRES_COLS[:]                                                    # presencias detectadas
GEO_DUM_BASELINE    = DUM_COMUNA + DUM_REGION                                         # todas las geo dummies
BIN_COLS_BASELINE   = OTRAS_BIN[:]                                                    # otras binarias 0/1 (si las hubiera)

# LISTAS PARA VARIANTE CON EMPRESA
NUM_COLS_EMPRESA    = [c for c in NUMERIC_CORE if c in df_empresa.columns]            # numéricas presentes
PRES_COLS_EMPRESA   = [c for c in PRES_COLS if c in df_empresa.columns]               # presencias en el set con empresa
GEO_DUM_EMPRESA     = [c for c in DUM_COMUNA + DUM_REGION if c in df_empresa.columns] # geo dummies presentes
EMP_BIN_COLS        = [c for c in EMP_BIN if c in df_empresa.columns]                 # TieneEmpresa / EsConvenio
EMP_DUM_COLS        = EMP_DUM[:]                                                      # Empresa_grp_* detectadas
BIN_COLS_EMPRESA    = [c for c in OTRAS_BIN if c in df_empresa.columns]               # otras binarias 0/1 también en este set

# Muestra un resumen compacto de las listas (para revisar a ojo)
def _resumen_listas(nombre, lst, max_show=12):
    vista = lst[:max_show] + (["… (+)"] if len(lst) > max_show else [])
    print(f"{nombre:24s} (n={len(lst):>3}):", vista)

print("\n=== LISTAS DE FEATURES — BASELINE (sin Empresa) ===")
_resumen_listas("NUM_COLS_BASELINE",  NUM_COLS_BASELINE)
_resumen_listas("PRES_COLS_BASELINE", PRES_COLS_BASELINE)
_resumen_listas("GEO_DUM_BASELINE",   GEO_DUM_BASELINE)
_resumen_listas("BIN_COLS_BASELINE",  BIN_COLS_BASELINE)

print("\n=== LISTAS DE FEATURES — CON EMPRESA ===")
_resumen_listas("NUM_COLS_EMPRESA",   NUM_COLS_EMPRESA)
_resumen_listas("PRES_COLS_EMPRESA",  PRES_COLS_EMPRESA)
_resumen_listas("GEO_DUM_EMPRESA",    GEO_DUM_EMPRESA)
_resumen_listas("EMP_BIN_COLS",       EMP_BIN_COLS)
_resumen_listas("EMP_DUM_COLS",       EMP_DUM_COLS)
_resumen_listas("BIN_COLS_EMPRESA",   BIN_COLS_EMPRESA)

# ────────────────────────────────────────────────────────────
# 4) Mini-checks ligeros (no son "tests duros", solo semáforo)
# ────────────────────────────────────────────────────────────
print("\n=== MINI-CHECKS DE AUDITORÍA ===")
print("Presencias = 4 ?                    ", "OK ✅" if set(PRES_COLS)==set(PRES_ESPERADAS) else "Revisar ⚠️")
print("Numéricas clave faltantes en Base ? ", "Ninguna ✅" if len(FALTAN_NUM)==0 else f"Faltan: {FALTAN_NUM} ⚠️")
print("¿Hay dummies de Comuna?             ", "Sí ✅" if len(DUM_COMUNA)>0 else "No ⚠️")
print("¿Hay dummies de Región?             ", "Sí ✅" if len(DUM_REGION)>0 else "No ⚠️")
print("¿Se detectan EMP_BIN en Empresa?    ", "Sí ✅" if len(EMP_BIN)>0 else "No ⚠️ (esperado si no aplica)")
print("¿Se detectan EMP_DUM en Empresa?    ", "Sí ✅" if len(EMP_DUM)>0 else "No ⚠️ (según corte)")


In [ ]:
# ============================================================
# PASO 2 — ESTANDARIZACIÓN Y PIPELINES (NB-03)
# ============================================================
# Objetivo: preparar los datasets para clustering.
# Creamos un ColumnTransformer que estandariza numéricas y deja
# tal cual (passthrough) presencias, dummies y flags binarios.

from sklearn.preprocessing import StandardScaler   # escalador por defecto
from sklearn.compose import ColumnTransformer      # permite aplicar transformaciones distintas por columna
from sklearn.pipeline import Pipeline              # armar pipeline de pasos consecutivos

# ────────────────────────────────────────────────────────────
# 1) Definimos transformador para NUMÉRICAS
# ────────────────────────────────────────────────────────────
scaler = StandardScaler()   # escalará cada numérica a media=0 y std=1

# ────────────────────────────────────────────────────────────
# 2) Pipeline para VARIANTE BASELINE (sin Empresa)
# ────────────────────────────────────────────────────────────
ct_base = ColumnTransformer(
    transformers=[
        ("num", scaler, NUM_COLS_BASELINE),        # aplica StandardScaler a las numéricas
        ("pres", "passthrough", PRES_COLS_BASELINE),  # deja pasar las presencias (0/1)
        ("geo", "passthrough", GEO_DUM_BASELINE),     # deja pasar dummies geográficas
        ("bin", "passthrough", BIN_COLS_BASELINE)     # deja pasar otras binarias 0/1
    ],
    remainder="drop"  # cualquier columna no listada se descarta
)

pipeline_base = Pipeline([
    ("transform", ct_base)   # paso único por ahora (solo transformar)
])

# Ajustamos (fit) y transformamos (transform) el dataset baseline
X_base_ready = pipeline_base.fit_transform(df_base)

print("=== VARIANTE BASELINE (sin Empresa) ===")
print("Shape original:", df_base.shape)                  # muestra shape original
print("Shape transformado:", X_base_ready.shape)         # filas deben coincidir; cols = suma de todas las listas
print("NaNs en transformado:", np.isnan(X_base_ready).sum())  # NaNs deberían ser 0
print()

# ────────────────────────────────────────────────────────────
# 3) Pipeline para VARIANTE CON EMPRESA
# ────────────────────────────────────────────────────────────
ct_emp = ColumnTransformer(
    transformers=[
        ("num", scaler, NUM_COLS_EMPRESA),           # StandardScaler a numéricas
        ("pres", "passthrough", PRES_COLS_EMPRESA),  # presencias
        ("geo", "passthrough", GEO_DUM_EMPRESA),     # dummies geo
        ("emp_bin", "passthrough", EMP_BIN_COLS),    # TieneEmpresa, EsConvenio
        ("emp_dum", "passthrough", EMP_DUM_COLS),    # Empresa_grp_*
        ("bin", "passthrough", BIN_COLS_EMPRESA)     # otras binarias 0/1
    ],
    remainder="drop"
)

pipeline_emp = Pipeline([
    ("transform", ct_emp)
])

X_emp_ready = pipeline_emp.fit_transform(df_empresa)

print("=== VARIANTE CON EMPRESA ===")
print("Shape original:", df_empresa.shape)
print("Shape transformado:", X_emp_ready.shape)
print("NaNs en transformado:", np.isnan(X_emp_ready).sum())
print()

# ────────────────────────────────────────────────────────────
# 4) Mini-checks ligeros
# ────────────────────────────────────────────────────────────
print("=== MINI-CHECKS ===")
print("Baseline: filas coinciden?", X_base_ready.shape[0] == df_base.shape[0])
print("Empresa: filas coinciden?", X_emp_ready.shape[0] == df_empresa.shape[0])
print("¿Baseline sin NaNs?", np.isnan(X_base_ready).sum() == 0)
print("¿Con Empresa sin NaNs?", np.isnan(X_emp_ready).sum() == 0)


In [ ]:
# ============================================================
# PASO 2b — REVISIÓN DE DISTRIBUCIÓN (numéricas clave)
# ============================================================
import matplotlib.pyplot as plt

# ────────────────────────────────────────────────────────────
# 1) Seleccionar las columnas numéricas clave del baseline
# ────────────────────────────────────────────────────────────
df_num = df_base[NUM_COLS_BASELINE].copy()    # subset solo con numéricas
print("Shape numéricas (baseline):", df_num.shape)

# ────────────────────────────────────────────────────────────
# 2) Estadísticos descriptivos extendidos
# ────────────────────────────────────────────────────────────
# Usamos .describe() para percentiles básicos, y agregamos p90/p99
desc = df_num.describe(percentiles=[0.9, 0.99]).T   # .T = transpuesta, filas=variables
print("=== ESTADÍSTICOS DE NUMÉRICAS (Baseline) ===")
print(desc[["mean","50%","90%","99%","min","max"]])  # mostramos solo columnas clave

# ────────────────────────────────────────────────────────────
# 3) Histogramas para cada columna numérica
# ────────────────────────────────────────────────────────────
for col in NUM_COLS_BASELINE:
    plt.figure(figsize=(6,4))                           # tamaño del gráfico
    plt.hist(df_num[col], bins=50, edgecolor="black")   # histograma con 50 bins
    plt.title(f"Distribución de {col}")                 # título con nombre de variable
    plt.xlabel(col)                                     # eje x
    plt.ylabel("Frecuencia")                            # eje y
    plt.grid(True, linestyle="--", alpha=0.6)           # rejilla ligera
    plt.show()


In [ ]:
# ============================================================
# PASO 2c — PIPELINE MIXTO (Standard + Robust + Passthrough)
# ============================================================
# Objetivo: escalar cada grupo de variables con la técnica adecuada
# y dejar listas las matrices X para clustering (sin entrenar aún).

from sklearn.preprocessing import StandardScaler, RobustScaler   # dos escaladores
from sklearn.compose import ColumnTransformer                    # mezcla de transformaciones por columna
from sklearn.pipeline import Pipeline                            # pipeline para encadenar pasos
import numpy as np                                               # chequeos de NaN

# ────────────────────────────────────────────────────────────
# 1) Definimos qué columnas van con cada tratamiento
#    (partimos de NUM_COLS_* que creaste en el Paso 1)
# ────────────────────────────────────────────────────────────

# a) StandardScaler → variables aproximadamente “normales”
STD_COLS_CANON = ["Edad"]                                        # en tu análisis, Edad se comporta razonable

# b) RobustScaler → variables con colas largas / muchos ceros
ROB_COLS_CANON = [
    "CantPptos", "CantPptosAbo", "CantPptosAvan",                # conteos sesgados
    "TotPptos_l1p", "TotPptosAbo_l1p", "TotPptosAvan_l1p",       # montos (log1p) con masa en 0 y colas
    "TicketPromPpto_l1p"
]

# c) Passthrough → ya normalizada (0–1), no necesita escalar
PASS_COLS_CANON = ["PctCumplimiento"]

# Nota: Por seguridad, intersectamos con las columnas PRESENTES en cada dataset
def _split_numeric_lists(cols_present):
    """Devuelve tres listas (std, rob, pass) válidas para el dataset dado."""
    std_cols  = [c for c in STD_COLS_CANON  if c in cols_present]
    rob_cols  = [c for c in ROB_COLS_CANON  if c in cols_present]
    pass_cols = [c for c in PASS_COLS_CANON if c in cols_present]
    # Si por alguna razón falta una numérica esperada, no crashea; simplemente no la usa.
    return std_cols, rob_cols, pass_cols

# ────────────────────────────────────────────────────────────
# 2) Construimos el ColumnTransformer para CADA VARIANTE
#    (baseline sin Empresa y con Empresa)
# ────────────────────────────────────────────────────────────

# 2.1) VARIANTE BASELINE (sin Empresa)
std_cols_base, rob_cols_base, pass_cols_base = _split_numeric_lists(NUM_COLS_BASELINE)

ct_base_mixto = ColumnTransformer(
    transformers=[
        ("num_std", StandardScaler(), std_cols_base),            # escala Edad (u otras que asignes)
        ("num_rob", RobustScaler(),   rob_cols_base),            # escala montos y conteos sesgados
        ("num_pas", "passthrough",    pass_cols_base),           # deja PctCumplimiento tal cual
        ("pres",    "passthrough",    PRES_COLS_BASELINE),       # presencias 0/1
        ("geo",     "passthrough",    GEO_DUM_BASELINE),         # dummies geográficas
        ("bin",     "passthrough",    BIN_COLS_BASELINE),        # otras binarias 0/1
    ],
    remainder="drop"                                             # descarta cualquier columna no listada
)

pipeline_base_mixto = Pipeline([
    ("transform", ct_base_mixto)                                 # por ahora, solo transformar
])

X_base_ready = pipeline_base_mixto.fit_transform(df_base)        # fit+transform para baseline

print("=== VARIANTE BASELINE (mixto) ===")
print("Original (filas, cols):", df_base.shape)
print("Transformado (filas, cols):", X_base_ready.shape)
print("NaNs en transformado:", int(np.isnan(X_base_ready).sum()))
print()

# 2.2) VARIANTE CON EMPRESA
std_cols_emp, rob_cols_emp, pass_cols_emp = _split_numeric_lists(NUM_COLS_EMPRESA)

ct_emp_mixto = ColumnTransformer(
    transformers=[
        ("num_std",  StandardScaler(), std_cols_emp),            # escala Edad (si está)
        ("num_rob",  RobustScaler(),   rob_cols_emp),            # escala montos y conteos sesgados
        ("num_pas",  "passthrough",    pass_cols_emp),           # PctCumplimiento
        ("pres",     "passthrough",    PRES_COLS_EMPRESA),       # presencias 0/1
        ("geo",      "passthrough",    GEO_DUM_EMPRESA),         # dummies geográficas
        ("emp_bin",  "passthrough",    EMP_BIN_COLS),            # TieneEmpresa / EsConvenio
        ("emp_dum",  "passthrough",    EMP_DUM_COLS),            # Empresa_grp_*
        ("bin",      "passthrough",    BIN_COLS_EMPRESA),        # otras binarias 0/1
    ],
    remainder="drop"
)

pipeline_emp_mixto = Pipeline([
    ("transform", ct_emp_mixto)
])

X_emp_ready = pipeline_emp_mixto.fit_transform(df_empresa)       # fit+transform para set con Empresa

print("=== VARIANTE CON EMPRESA (mixto) ===")
print("Original (filas, cols):", df_empresa.shape)
print("Transformado (filas, cols):", X_emp_ready.shape)
print("NaNs en transformado:", int(np.isnan(X_emp_ready).sum()))
print()

# ────────────────────────────────────────────────────────────
# 3) Mini-checks rápidos para tu tranquilidad
# ────────────────────────────────────────────────────────────
def _ok(flag): return "OK ✅" if flag else "Revisar ⚠️"

print("=== MINI-CHECKS ===")
print("Baseline: filas conservadas      :", _ok(X_base_ready.shape[0] == df_base.shape[0]))
print("Baseline: sin NaNs               :", _ok(np.isnan(X_base_ready).sum() == 0))
print("Con Empresa: filas conservadas   :", _ok(X_emp_ready.shape[0] == df_empresa.shape[0]))
print("Con Empresa: sin NaNs            :", _ok(np.isnan(X_emp_ready).sum() == 0))

# (Opcional) Si quieres guardar las columnas usadas por transformador:
print("\nListas usadas (baseline):")
print("  STD :", std_cols_base)
print("  ROB :", rob_cols_base)
print("  PASS:", pass_cols_base)
print("Listas usadas (con empresa):")
print("  STD :", std_cols_emp)
print("  ROB :", rob_cols_emp)
print("  PASS:", pass_cols_emp)


In [ ]:
# ============================================================
# PASO 3 — BARRIDO DE K (2..10) con K-Means — 100% comentado
# ============================================================

# 1) Imports necesarios para clustering, métricas, gráficos y tabla
from sklearn.cluster import KMeans              # algoritmo K-Means para agrupar
from sklearn.metrics import silhouette_score   # métrica para evaluar separación entre clusters
import matplotlib.pyplot as plt                # gráficos (usamos matplotlib, no seaborn)
import numpy as np                             # utilidades numéricas
import pandas as pd                            # para armar tabla resumen

# 2) Validaciones suaves: asegurarnos de que las matrices transformadas existen
assert 'X_base_ready' in globals(), "X_base_ready no existe. Ejecuta el Paso 2c (pipeline mixto) primero."
assert 'X_emp_ready'  in globals(), "X_emp_ready no existe. Ejecuta el Paso 2c (pipeline mixto) primero."

# 3) Definimos una función que recorre K entre k_min y k_max y guarda las métricas
def barrido_k(X, k_min=2, k_max=10, random_state=42):        # recibe matriz X ya transformada y rango de K
    inertias = []                                            # lista vacía para la inercia (SSE) por cada K
    silhouettes = []                                         # lista vacía para el silhouette por cada K
    for k in range(k_min, k_max + 1):                        # iteramos K = 2,3,...,k_max
        km = KMeans(                                         # instanciamos el modelo K-Means
            n_clusters=k,                                    # número de clusters a probar
            random_state=random_state,                       # semilla para reproducibilidad
            n_init=10                                        # nº de reinicios para evitar malos mínimos locales
        )
        labels = km.fit_predict(X)                           # ajusta el modelo y devuelve etiquetas para cada fila
        inertias.append(km.inertia_)                         # guardamos la inercia (SSE dentro de clusters)
        sil = silhouette_score(X, labels) if k > 1 else np.nan  # silhouette requiere al menos 2 clusters
        silhouettes.append(sil)                              # guardamos el silhouette de este K
    return inertias, silhouettes                             # devolvemos ambas listas

# 4) Ejecutamos el barrido para la variante BASELINE (sin Empresa)
inertias_base, sils_base = barrido_k(                        # llamamos la función de barrido
    X=X_base_ready,                                          # matriz X ya escalada/transformada (baseline)
    k_min=2,                                                 # K mínimo a evaluar
    k_max=10,                                                # K máximo a evaluar
    random_state=42                                          # semilla fija
)

# 5) Ejecutamos el barrido para la variante CON EMPRESA
inertias_emp, sils_emp = barrido_k(                          # repetimos el barrido para la otra variante
    X=X_emp_ready,                                           # matriz X con Empresa/Convenio
    k_min=2,                                                 # mismo rango de K
    k_max=10,                                                # mismo rango de K
    random_state=42                                          # misma semilla
)

# 6) Construimos la lista de valores K evaluados (2..10)
K_range = list(range(2, 11))                                 # lista [2,3,...,10] para alinear con resultados

# 7) Graficamos la CURVA DEL CODO (Inertia) para comparar ambas variantes
plt.figure(figsize=(12, 5))                                  # creamos una figura de 12x5 pulgadas
plt.subplot(1, 2, 1)                                         # primer panel (izquierda) de 1 fila x 2 columnas
plt.plot(K_range, inertias_base, "o-", label="Baseline")     # trazamos inercia de baseline (marcador círculo)
plt.plot(K_range, inertias_emp, "s--", label="Con Empresa")  # trazamos inercia con empresa (marcador cuadrado)
plt.xlabel("Número de clusters (K)")                         # etiqueta eje X
plt.ylabel("Inertia (SSE)")                                  # etiqueta eje Y
plt.title("Curva del codo (SSE)")                            # título del gráfico
plt.legend()                                                 # mostramos leyenda para distinguir variantes
plt.grid(True, linestyle="--", alpha=0.6)                    # rejilla suave para leer mejor

# 8) Graficamos el SILHOUETTE para comparar ambas variantes
plt.subplot(1, 2, 2)                                         # segundo panel (derecha)
plt.plot(K_range, sils_base, "o-", label="Baseline")         # silhouette baseline
plt.plot(K_range, sils_emp, "s--", label="Con Empresa")      # silhouette con empresa
plt.xlabel("Número de clusters (K)")                         # etiqueta eje X
plt.ylabel("Silhouette score")                               # etiqueta eje Y
plt.title("Comparación de silhouette")                       # título del gráfico
plt.legend()                                                 # leyenda para distinguir líneas
plt.grid(True, linestyle="--", alpha=0.6)                    # rejilla suave

plt.tight_layout()                                           # ajusta espacios entre subplots automáticamente
plt.show()                                                   # muestra los gráficos

# 9) Armamos una TABLA RESUMEN con todos los valores para inspección fina
tabla_resumen = pd.DataFrame({                               # construimos un DataFrame con columnas:
    "K": K_range,                                            # K evaluado
    "Inertia_Base": inertias_base,                           # inercia para baseline
    "Silhouette_Base": sils_base,                            # silhouette para baseline
    "Inertia_Emp": inertias_emp,                             # inercia para con empresa
    "Silhouette_Emp": sils_emp                               # silhouette para con empresa
})

# 10) Calculamos sugerencias automáticas (no decide, solo guía)
k_mejor_sil_base = int(tabla_resumen.loc[tabla_resumen["Silhouette_Base"].idxmax(), "K"])  # K con mayor silhouette (baseline)
k_mejor_sil_emp  = int(tabla_resumen.loc[tabla_resumen["Silhouette_Emp"].idxmax(), "K"])   # K con mayor silhouette (con empresa)

# 11) Mostramos la tabla y las sugerencias por silhouette
print("=== TABLA RESUMEN — BARRIDO K ===")                  # cabecera de la tabla
display(tabla_resumen)                                     # mostramos la tabla en el notebook
print(f"Sugerencia por silhouette (Baseline): K = {k_mejor_sil_base}")  # sugerencia baseline
print(f"Sugerencia por silhouette (Con Empresa): K = {k_mejor_sil_emp}")# sugerencia con empresa

# 12) (Opcional) Heurística simple de "codo": diferencia relativa de SSE
#     Nota: esto NO es una elección automática. Solo te indica dónde el SSE deja de bajar fuerte.
sse_base = np.array(inertias_base)                          # convertimos



In [ ]:
# ============================================================
# PASO 4 — ENTRENAMIENTO FINAL K=3
# ============================================================
from sklearn.cluster import KMeans

# ────────────────────────────────────────────────────────────
# 1) Definimos función auxiliar para entrenar y devolver etiquetas + modelo
# ────────────────────────────────────────────────────────────
def entrenar_kmeans(X, n_clusters=3, random_state=42):
    """
    Entrena KMeans con K clusters y devuelve:
    - modelo entrenado
    - etiquetas de cluster para cada fila
    """
    km = KMeans(
        n_clusters=n_clusters,       # número de clusters
        random_state=random_state,   # semilla reproducible
        n_init=10                    # nº de reinicios (reduce riesgo de mínimos locales)
    )
    labels = km.fit_predict(X)       # entrena y asigna cluster a cada fila
    return km, labels

# ────────────────────────────────────────────────────────────
# 2) Entrenamos con VARIANTE BASELINE (sin Empresa)
# ────────────────────────────────────────────────────────────
km_base, labels_base = entrenar_kmeans(X_base_ready, n_clusters=3)

# Añadimos etiquetas al DataFrame de IDs (para vincular con paciente)
df_ids["cluster_baseline"] = labels_base

print("=== VARIANTE BASELINE ===")
print("Tamaño por cluster:")
print(pd.Series(labels_base).value_counts().sort_index())   # conteo de pacientes por cluster

# ────────────────────────────────────────────────────────────
# 3) Entrenamos con VARIANTE CON EMPRESA
# ────────────────────────────────────────────────────────────
km_emp, labels_emp = entrenar_kmeans(X_emp_ready, n_clusters=3)

# Añadimos etiquetas también
df_ids["cluster_con_empresa"] = labels_emp

print("\n=== VARIANTE CON EMPRESA ===")
print("Tamaño por cluster:")
print(pd.Series(labels_emp).value_counts().sort_index())

# ────────────────────────────────────────────────────────────
# 4) Guardamos centroides (en espacio transformado)
# ────────────────────────────────────────────────────────────
centroides_base = pd.DataFrame(
    km_base.cluster_centers_,              # centroides (matriz K x nº_features)
    columns=ct_base_mixto.get_feature_names_out(),  # nombres de features después del transformer
)
centroides_base["cluster"] = centroides_base.index

centroides_emp = pd.DataFrame(
    km_emp.cluster_centers_,
    columns=ct_emp_mixto.get_feature_names_out(),
)
centroides_emp["cluster"] = centroides_emp.index

print("\n=== Centroides (Baseline, primeras columnas) ===")
display(centroides_base.iloc[:,:10])   # mostramos solo 10 primeras cols para no saturar

print("\n=== Centroides (Con Empresa, primeras columnas) ===")
display(centroides_emp.iloc[:,:10])

# ────────────────────────────────────────────────────────────
# 5) Perfiles en espacio original (medianas por cluster)
# ────────────────────────────────────────────────────────────
# Baseline
perfil_base = df_base.copy()
perfil_base["cluster_baseline"] = labels_base
perfil_base = perfil_base.groupby("cluster_baseline").median(numeric_only=True)

# Con Empresa
perfil_emp = df_empresa.copy()
perfil_emp["cluster_con_empresa"] = labels_emp
perfil_emp = perfil_emp.groupby("cluster_con_empresa").median(numeric_only=True)

print("\n=== Perfiles por mediana (Baseline) ===")
display(perfil_base[NUM_COLS_BASELINE + PRES_COLS_BASELINE].round(2))  # mostramos numéricas+presencias

print("\n=== Perfiles por mediana (Con Empresa) ===")
display(perfil_emp[NUM_COLS_EMPRESA[:10]].round(2))  # mostramos primeras 10 numéricas para ver patrón

# ────────────────────────────────────────────────────────────
# 6) Export opcional de resultados (para NB-04 y validación)
# ────────────────────────────────────────────────────────────
# IDs + clusters
df_ids.to_csv(ROOT / "data" / "processed" / "activos_ids_v2_plus_clustered.csv", index=False)

# Centroides
centroides_base.to_csv(ROOT / "data" / "processed" / "clusters_centroids_baseline.csv", index=False)
centroides_emp.to_csv(ROOT / "data" / "processed" / "clusters_centroids_con_empresa.csv", index=False)

# Perfiles
perfil_base.to_csv(ROOT / "data" / "processed" / "cluster_profiles_baseline.csv")
perfil_emp.to_csv(ROOT / "data" / "processed" / "cluster_profiles_con_empresa.csv")


In [ ]:
# ============================================================
# PASO 5 — PERFILADO E INSIGHTS ACCIONABLES
# ============================================================
# Objetivo: traducir los perfiles de clusters en un resumen humano,
# con insights útiles para la gestión (retención, reactivación, depuración).

# ────────────────────────────────────────────────────────────
# 1) Preparamos un diccionario con "narrativas" de cada cluster
# ────────────────────────────────────────────────────────────
insights_baseline = {
    0: {
        "nombre": "Uso bajo / abandono latente",
        "perfil": "Pacientes de ~35 años, con 1 presupuesto de monto medio, cumplimiento parcial (≈67%), sin atenciones en últimos meses.",
        "acciones": [
            "Campañas de reactivación (recordatorios, WhatsApp, email).",
            "Ofertas de control preventivo o chequeos básicos.",
            "Identificar causas de abandono (precio, experiencia, distancia)."
        ]
    },
    1: {
        "nombre": "Alta frecuencia / alta conversión",
        "perfil": "Pacientes jóvenes (~26 años), con ~5 presupuestos, alta tasa de cumplimiento (≈80%), con atenciones en últimos 6 meses.",
        "acciones": [
            "Enfocar en retención y fidelización (programas de membresía, descuentos por continuidad).",
            "Promover recomendaciones y derivaciones (traer amigos/familia).",
            "Monitorear satisfacción para evitar pérdida de este segmento clave."
        ]
    },
    2: {
        "nombre": "Inactivos / fantasma",
        "perfil": "Pacientes de ~38 años, sin presupuestos activos, sin pagos ni atenciones recientes.",
        "acciones": [
            "Revisar calidad del registro (¿pacientes duplicados, históricos?).",
            "Campañas muy ligeras (email masivo o SMS) → bajo costo.",
            "Probable depuración de base si no responden."
        ]
    }
}

# ────────────────────────────────────────────────────────────
# 2) Imprimimos en formato legible (como brief para gerente)
# ────────────────────────────────────────────────────────────
print("=== INSIGHTS CLUSTERS (Baseline) ===\n")
for cluster, info in insights_baseline.items():
    print(f"Cluster {cluster} — {info['nombre']}")
    print(f"Perfil: {info['perfil']}")
    print("Acciones sugeridas:")
    for act in info["acciones"]:
        print(f"  - {act}")
    print("\n")

# ────────────────────────────────────────────────────────────
# 3) (Opcional) Export a un .md (Markdown) para usar directo en Notion/GitHub
# ────────────────────────────────────────────────────────────
output_md = ROOT / "reports" / "cluster_insights_baseline.md"
output_md.parent.mkdir(parents=True, exist_ok=True)

with open(output_md, "w", encoding="utf-8") as f:
    f.write("# Insights Clusters (Baseline)\n\n")
    for cluster, info in insights_baseline.items():
        f.write(f"## Cluster {cluster} — {info['nombre']}\n")
        f.write(f"**Perfil:** {info['perfil']}\n\n")
        f.write("**Acciones sugeridas:**\n")
        for act in info["acciones"]:
            f.write(f"- {act}\n")
        f.write("\n")


In [ ]:
# ============================================================
# PASO 5b — COMPARATIVA BASELINE vs CON EMPRESA
# ============================================================
# Objetivo: mostrar si incluir Empresa/Convenio aporta algo nuevo al clustering.
# Vamos a comparar:
#   1. Tamaños de clusters.
#   2. Perfiles (medianas).
#   3. Conclusión narrativa.

# ────────────────────────────────────────────────────────────
# 1) Conteo de pacientes por cluster en cada variante
# ────────────────────────────────────────────────────────────
conteos_base = pd.Series(labels_base).value_counts().sort_index()
conteos_emp  = pd.Series(labels_emp).value_counts().sort_index()

print("=== Conteo de pacientes por cluster ===")
print("Baseline (sin Empresa):")
print(conteos_base)
print("\nCon Empresa:")
print(conteos_emp)
print()

# ────────────────────────────────────────────────────────────
# 2) Diferencia absoluta de conteos
# ────────────────────────────────────────────────────────────
diff_conteos = conteos_emp - conteos_base
print("Diferencia en nº de pacientes (Con Empresa - Baseline):")
print(diff_conteos)
print()

# ────────────────────────────────────────────────────────────
# 3) Comparar perfiles de mediana en variables clave
# ────────────────────────────────────────────────────────────
# Seleccionamos las columnas numéricas comunes
cols_clave = ["Edad","CantPptos","TotPptos_l1p","PctCumplimiento","TicketPromPpto_l1p"]

comparativa_perfiles = pd.concat([
    perfil_base[cols_clave].add_suffix("_Base"),
    perfil_emp[cols_clave].add_suffix("_Emp")
], axis=1)

print("=== Comparativa de perfiles (medianas) ===")
display(comparativa_perfiles.round(2))

# ────────────────────────────────────────────────────────────
# 4) Conclusión narrativa
# ────────────────────────────────────────────────────────────
print("=== Conclusión narrativa ===")
print("La distribución de pacientes por cluster es prácticamente idéntica en ambas variantes.")
print("Los perfiles (edad, nº de presupuestos, ticket promedio, % cumplimiento) son casi iguales.")
print("→ Esto indica que las variables de Empresa/Convenio NO aportan diferenciación adicional significativa.")
print("→ La dinámica de uso clínico y presupuestos domina el agrupamiento.")
